In [7]:
import pandas as pd
import numpy as np
import os, sys
from IPython.display import display
from IPython.display import Image as _Imgdis
from numpy import asarray
from matplotlib import pyplot

#d = {'Open1': [0], 'High': [0],'Low1': [0], 'Close': [0],'Pred_Open': [0], 'Pred_High': [0],'Pred_Low': [0], 'Pred_Close': [0]}
#output = pd.DataFrame(data=d)
#output = pd.read_csv("Export_data.csv", header=0)
#output["open_change"] = (output["Open"] / output["Open"].shift(-1)) -1
#output["high_change"] = (output["High"] / output["High"].shift(-1)) -1
#output["low_change"] = (output["Low"] / output["Low"].shift(-1)) -1
#output["close_change"] = (output["Close"] / output["Close"].shift(-1)) - 1
#output.to_csv (r'C:\Users\TH24\Desktop\Code\Code\Bitcoin-Bot\Export_data.csv', index = False, header=True)

df = pd.read_csv("Bittrex_BTCUSD_1h.csv", header=1)
df2 = pd.read_csv("Bittrex_ETHUSD_1h.csv",header=1)
df3 = pd.read_csv("Bittrex_LTCUSD_1h.csv",header=1)

df = pd.concat([df, df2, df3])
df.reset_index(drop=True, inplace=True)

df["open_change"] = (df["Open"] / df["Open"].shift(-1)) -1
df["high_change"] = (df["High"] / df["High"].shift(-1)) -1
df["low_change"] = (df["Low"] / df["Low"].shift(-1)) -1
df["close_change"] = (df["Close"] / df["Close"].shift(-1)) - 1

df = df.drop("Unix Timestamp", axis=1)
df = df.drop("Date", axis=1)
df = df.drop("Volume BTC", axis=1)
df = df.drop("Volume ETH", axis=1)
df = df.drop("Volume LTC", axis=1)
df = df.drop("Volume USD", axis=1)
df.drop(df.tail(8).index,inplace=True)

df = df.dropna()

df

,Symbol,Open,High,Low,Close,open_change,high_change,low_change,close_change
0,BTCUSD,18168.59,18390.00,18100.00,18194.27,0.002070,0.011144,0.001045,0.001413
1,BTCUSD,18131.05,18187.32,18081.10,18168.59,-0.003849,-0.002197,0.001187,0.002070
2,BTCUSD,18201.11,18227.37,18059.67,18131.05,0.013638,-0.000418,0.006381,-0.003849
3,BTCUSD,17956.22,18234.99,17945.16,18201.11,-0.000905,0.011471,-0.000496,0.013638
4,BTCUSD,17972.49,18028.19,17954.06,17956.22,-0.000483,-0.001608,0.000862,-0.000905
...,...,...,...,...,...,...,...,...,...
88795,LTCUSD,40.05,40.05,40.05,40.05,-0.013547,-0.013547,0.011364,0.000000
88796,LTCUSD,40.60,40.60,39.60,40.05,0.038363,-0.013126,0.036649,-0.013547
88797,LTCUSD,39.10,41.14,38.20,40.60,-0.022744,0.013800,-0.023018,0.038363
88798,LTCUSD,40.01,40.58,39.10,39.10,-0.028412,-0.014570,-0.022500,-0.022744


In [8]:

X = []
Y = []

changes = df[["open_change", "high_change", "low_change", "close_change"]].to_numpy()


for i in range(0, len(changes) - 20):
    Y.append(changes[i])
    X.append(np.array(changes[i+1:i+21][::-1]))
    
X = np.array(X).reshape(-1, 20, 4)
Y = np.array(Y)
print(X.shape)
print(Y.shape);

(88780, 20, 4)
(88780, 4)


In [9]:
from keras.models import Sequential 
from keras.layers import LSTM
from keras.models import load_model

model = Sequential()
model.add(LSTM(4, input_shape=(20, 4)))


model.compile(optimizer="rmsprop", loss="mse")
model.fit(X, Y, batch_size=10, epochs=100)

Epoch 1/100
8878/8878 [==============================] - 30s 3ms/step - loss: 3.3683e-04
Epoch 2/100
8878/8878 [==============================] - 28s 3ms/step - loss: 3.1712e-04
Epoch 3/100
8878/8878 [==============================] - 29s 3ms/step - loss: 3.1525e-04
Epoch 4/100
8878/8878 [==============================] - 28s 3ms/step - loss: 3.1440e-04
Epoch 5/100
8878/8878 [==============================] - 26s 3ms/step - loss: 3.1372e-04
Epoch 6/100
8878/8878 [==============================] - 27s 3ms/step - loss: 3.1334e-04
Epoch 7/100
8878/8878 [==============================] - 27s 3ms/step - loss: 3.1296e-04
Epoch 8/100
8878/8878 [==============================] - 27s 3ms/step - loss: 3.1273e-04
Epoch 9/100
8878/8878 [==============================] - 26s 3ms/step - loss: 3.1258e-04
Epoch 10/100
8878/8878 [==============================] - 26s 3ms/step - loss: 3.1244e-04
Epoch 11/100
8878/8878 [==============================] - 28s 3ms/step - loss: 3.1224e-04
Epoch 12/100
8878/8

In [10]:
predictions = model.predict(X)
df1 = pd.DataFrame(predictions, columns=['predictions_open', 'predictions_High', 'predictions_Low', 'predictions_Close'])
df.drop(df.tail(20).index,inplace=True)
model.save("Crypo_AI.hdf5")

df1['predictions_Close'].map(int)
df.insert(6, 'predictions_open', df1["predictions_open"])
df.insert(8, 'predictions_High', df1["predictions_High"])
df.insert(10, 'predictions_Low', df1["predictions_Low"])
df.insert(12, 'predictions_Close', df1["predictions_Close"])
df

,Symbol,Open,High,Low,Close,open_change,predictions_open,high_change,predictions_High,low_change,predictions_Low,close_change,predictions_Close
0,BTCUSD,18168.59,18390.00,18100.00,18194.27,0.002070,0.001904,0.011144,0.002118,0.001045,0.000098,0.001413,-0.000761
1,BTCUSD,18131.05,18187.32,18081.10,18168.59,-0.003849,-0.004047,-0.002197,-0.001164,0.001187,-0.001272,0.002070,-0.000393
2,BTCUSD,18201.11,18227.37,18059.67,18131.05,0.013638,0.013562,-0.000418,0.002223,0.006381,0.008897,-0.003849,-0.001558
3,BTCUSD,17956.22,18234.99,17945.16,18201.11,-0.000905,-0.001123,0.011471,-0.000120,-0.000496,-0.003353,0.013638,-0.000218
4,BTCUSD,17972.49,18028.19,17954.06,17956.22,-0.000483,-0.000717,-0.001608,-0.000643,0.000862,-0.002611,-0.000905,-0.000243
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88775,LTCUSD,38.41,40.65,37.71,39.79,0.005497,0.005528,0.038845,-0.002471,0.000000,0.006242,0.035928,0.000005
88776,LTCUSD,38.20,39.13,37.71,38.41,-0.007792,-0.007860,0.008245,-0.000560,-0.012827,-0.009726,0.005497,0.001193
88777,LTCUSD,38.50,38.81,38.20,38.20,-0.012821,-0.012744,-0.030477,-0.014040,-0.007792,-0.011033,-0.007792,0.005726
88778,LTCUSD,39.00,40.03,38.50,38.50,-0.041062,-0.040587,-0.015736,-0.029355,-0.012821,-0.014910,-0.012821,0.004284


In [12]:
import sched, time
import requests
import re

s = sched.scheduler(time.time, time.sleep)

def loop(sc): 
    print("Doing stuff...")

    X_New = []
    Y_New = []
    resp = requests.get('https://api.coingecko.com/api/v3/coins/bitcoin/ohlc?vs_currency=eur&days=1').json()
    firstElement = resp[:1]
    string = " ".join(str(x) for x in firstElement)
    string = string.replace('[', '')
    string =string.replace(']', '')
    string = string.replace(',', '')
    string = string.split()
    print(string[0])

    output_read = pd.read_csv("Export_data.csv", header=0)
    changes_New = output_read[["open_change", "high_change", "low_change", "close_change"]].to_numpy()

    for i in range(0, 20):
        X_New.append(np.array(changes_New[i+1:i+21][::-1]))
    
    X_New = np.array(X_New).reshape(-1, 20, 4)


    predictions_New = model.predict(X_New)



    output_read.loc[0, 'Time'] = string[0]
    output_read.loc[0, 'Open'] = float(string[1])
    output_read.loc[0, 'High'] = float(string[2])
    output_read.loc[0, 'Low'] = float(string[3])
    output_read.loc[0, 'Close'] = float(string[4])


    output_read.loc[0, 'open_change'] = output_read.loc[0, 'Open'] / output_read.loc[1, 'Open']
    output_read.loc[0, 'high_change'] = output_read.loc[0, 'High'] / output_read.loc[1, 'High']
    output_read.loc[0, 'low_change'] = output_read.loc[0, 'Low'] / output_read.loc[1, 'Low']
    output_read.loc[0, 'close_change'] = output_read.loc[0, 'Close'] / output_read.loc[1, 'Close']




    output_read.loc[-1] = ['0', '0','0','0','0',predictions_New[0][0], predictions_New[0][1] ,predictions_New[0][2] ,predictions_New[0][3] ,'0','0','0','0']  
    output_read.index = output_read.index + 1  
    output_read.sort_index(inplace=True) 


    output_read.to_csv (r'C:\Users\TH24\Desktop\Code\Code\Bitcoin-Bot\Export_data.csv', index = False, header=True)
   

    
    s.enter(1800, 1, loop, (sc,))

s.enter(1800, 1, loop, (s,))
s.run()

In [ ]:
import requests
import re
X_New = []
Y_New = []
resp = requests.get('https://api.coingecko.com/api/v3/coins/bitcoin/ohlc?vs_currency=eur&days=1').json()
firstElement = resp[:1]
string = " ".join(str(x) for x in firstElement)
string = string.replace('[', '')
string =string.replace(']', '')
string = string.replace(',', '')
string = string.split()
print(string[0])

output_read = pd.read_csv("Export_data.csv", header=0)
changes_New = output_read[["open_change", "high_change", "low_change", "close_change"]].to_numpy()

for i in range(0, 20):
    X_New.append(np.array(changes_New[i+1:i+21][::-1]))
    
X_New = np.array(X_New).reshape(-1, 20, 4)


predictions_New = model.predict(X_New)



# output_read.loc[0, 'Time'] = string[0]
# output_read.loc[0, 'Open'] = float(string[1])
# output_read.loc[0, 'High'] = float(string[2])
# output_read.loc[0, 'Low'] = float(string[3])
# output_read.loc[0, 'Close'] = float(string[4])


# output_read.loc[0, 'open_change'] = output_read.loc[0, 'Open'] / output_read.loc[1, 'Open']
# output_read.loc[0, 'high_change'] = output_read.loc[0, 'High'] / output_read.loc[1, 'High']
# output_read.loc[0, 'low_change'] = output_read.loc[0, 'Low'] / output_read.loc[1, 'Low']
# output_read.loc[0, 'close_change'] = output_read.loc[0, 'Close'] / output_read.loc[1, 'Close']




# output_read.loc[-1] = ['0', '0','0','0','0',predictions_New[0][0], predictions_New[0][1] ,predictions_New[0][2] ,predictions_New[0][3] ,'0','0','0','0']  
# output_read.index = output_read.index + 1  
# output_read.sort_index(inplace=True) 


# output_read.to_csv (r'C:\Users\TH24\Desktop\Code\Code\Bitcoin-Bot\Export_data.csv', index = False, header=True)

1635357600000


<ipython-input-11-30ab4e6937f6>:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X_New = np.array(X_New).reshape(-1, 20, 4)


ValueError: cannot reshape array of size 20 into shape (20,4)